<a href="https://colab.research.google.com/github/vkthakur90/jupyter_notebooks/blob/master/LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q gradio chromadb langchain langchainhub huggingface_hub python-dotenv

In [2]:
import langchain as lc
import langchain.embeddings as lc_embed
import langchain.document_loaders as lcdl
import langchain.text_splitter as lcts
import langchain.vectorstores as lcvs
import langchain.chains as lcchains
import langchain.hub as lchub

In [3]:
import os
import google.colab.files as gcf
import numpy as np
import pandas as pd
import dotenv as denv
import gradio as gr

# **Uploading environment files containing huggingface access token**

In [4]:
!ls

uploaded = gcf.upload()

for k, v in uploaded.items():
    with open(k, 'wb') as f:
        f.write(v)

!ls

flagged  hf.env  sample_data  story.txt


flagged  hf.env  sample_data  story.txt


In [5]:
denv.load_dotenv('hf.env')

True

In [6]:
text_loader = lcdl.TextLoader('story.txt')

# **Mention repository identity of hosted model**

In [7]:
hf_google_flan_t5_xxl = lc.HuggingFaceHub(repo_id = 'google/flan-t5-xxl')
hf_gpt2_xl = lc.HuggingFaceHub(repo_id = 'gpt2-xl')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [8]:
sentence_embeddings = lc_embed.HuggingFaceInferenceAPIEmbeddings(
    api_key = os.environ['HUGGINGFACEHUB_API_TOKEN'],
    model_name = 'sentence-transformers/all-MiniLM-L6-v2'
)

In [9]:
documents = text_loader.load()

In [10]:
text_splitter = lcts.RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap  = 10,
)

In [11]:
splits = text_splitter.split_documents(documents)

In [12]:
db = lcvs.Chroma.from_documents(splits, sentence_embeddings)

In [13]:
retriever = db.as_retriever(search_type="mmr")

# **Create prompt template**

In [14]:
template = """
You are a novel writer. Write response of question based on following context:

-----------------------------
Context : {context}
-----------------------------

{question}
"""

In [15]:
qa_prompt = lc.PromptTemplate.from_template(template)

In [16]:
qa_chain = lcchains.RetrievalQA.from_chain_type(
    llm = hf_google_flan_t5_xxl,
    retriever = retriever,
    return_source_documents = True,
    chain_type_kwargs={"prompt": qa_prompt}
)

In [17]:
def rag_interface(ques, history):
    prev_chat = " ".join([f"Question: {q} \nAnswer: {a} \n" for q, a in history])
    current_ques = f'\nQuestion: {ques} \nAnswer:'

    query = prev_chat + current_ques

    result = qa_chain({
        'query' : query
    })

    return result['result']

In [18]:
demo = gr.ChatInterface(fn=rag_interface)

In [19]:
demo.launch(share = True, debug = True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://02f3c2c682fa90b89e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://02f3c2c682fa90b89e.gradio.live
